In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, count, expr, lit, current_date, months_between, to_date, datediff

spark = SparkSession.builder.appName("RetailInventoryAnalysis").getOrCreate()

df = spark.read.option("header", True).option("inferSchema", True).csv("file:/Workspace/Shared/inventory_supply.csv")

df = df.withColumn("LastRestocked", to_date("LastRestocked"))
df.show()

+------+------------+-----------+----------+--------+------------+-------------+---------+---------+
|ItemID|    ItemName|   Category| Warehouse|StockQty|ReorderLevel|LastRestocked|UnitPrice| Supplies|
+------+------------+-----------+----------+--------+------------+-------------+---------+---------+
|  I001|      LED TV|Electronics|WarehouseA|      50|          20|   2024-03-15|    30000|   AVTech|
|  I002|      Laptop|Electronics|WarehouseB|      10|          15|   2024-04-01|    70000|TechWorld|
|  I003|Office Chair|  Furniture|WarehouseA|      40|          10|   2024-03-25|     6000|  ChairCo|
|  I004|Refrigerator| Appliances|WarehouseC|       5|          10|   2024-02-20|    25000| FreezeIt|
|  I005|     Printer|Electronics|WarehouseB|       3|           5|   2024-03-30|     8000|PrintFast|
+------+------------+-----------+----------+--------+------------+-------------+---------+---------+



Scenario 1: Inventory Alerting System

In [0]:
#2. Create a new column NeedsReorder = StockQty < ReorderLevel .
df = df.withColumn("NeedsReorder", col("StockQty") < col("ReorderLevel"))

#3. Create a view of all items that need restocking.
df.filter("NeedsReorder").createOrReplaceTempView("items_to_restock")
spark.sql("SELECT * FROM items_to_restock").show()

#4. Highlight warehouses with more than 2 such items.
df.filter("NeedsReorder").groupBy("Warehouse").count().filter("count > 2").show()

+------+------------+-----------+----------+--------+------------+-------------+---------+---------+------------+
|ItemID|    ItemName|   Category| Warehouse|StockQty|ReorderLevel|LastRestocked|UnitPrice| Supplies|NeedsReorder|
+------+------------+-----------+----------+--------+------------+-------------+---------+---------+------------+
|  I002|      Laptop|Electronics|WarehouseB|      10|          15|   2024-04-01|    70000|TechWorld|        true|
|  I004|Refrigerator| Appliances|WarehouseC|       5|          10|   2024-02-20|    25000| FreezeIt|        true|
|  I005|     Printer|Electronics|WarehouseB|       3|           5|   2024-03-30|     8000|PrintFast|        true|
+------+------------+-----------+----------+--------+------------+-------------+---------+---------+------------+

+---------+-----+
|Warehouse|count|
+---------+-----+
+---------+-----+



Scenario 2: Supplier Price Optimization

In [0]:
# 1. Group items by Supplier and compute average price.
avg_price_by_supplier = df.groupBy("Supplies").agg(avg("UnitPrice").alias("AvgSupplierPrice"))
avg_price_by_supplier.show()

# 2. Find which suppliers offer items below average price in their category.
avg_price_by_category = df.groupBy("Category").agg(avg("UnitPrice").alias("CategoryAvgPrice"))
df2 = df.join(avg_price_by_category, "Category")

df2 = df2.withColumn("IsBelowMarket", col("UnitPrice") < col("CategoryAvgPrice"))
df2.select("ItemName", "Supplies", "Category", "UnitPrice", "IsBelowMarket").show()


+---------+----------------+
| Supplies|AvgSupplierPrice|
+---------+----------------+
|   AVTech|         30000.0|
|TechWorld|         70000.0|
|PrintFast|          8000.0|
| FreezeIt|         25000.0|
|  ChairCo|          6000.0|
+---------+----------------+

+------------+---------+-----------+---------+-------------+
|    ItemName| Supplies|   Category|UnitPrice|IsBelowMarket|
+------------+---------+-----------+---------+-------------+
|      LED TV|   AVTech|Electronics|    30000|         true|
|      Laptop|TechWorld|Electronics|    70000|        false|
|Office Chair|  ChairCo|  Furniture|     6000|        false|
|Refrigerator| FreezeIt| Appliances|    25000|        false|
|     Printer|PrintFast|Electronics|     8000|         true|
+------------+---------+-----------+---------+-------------+



In [0]:
# 3. Tag suppliers with Good Deal if >50% of their items are below market average.
good_deal = df2.groupBy("Supplies") \
    .agg(expr("sum(case when IsBelowMarket then 1 else 0 end) / count(*)").alias("GoodDealRatio")) \
    .withColumn("Tag", when(col("GoodDealRatio") > 0.5, "Good Deal").otherwise("Average"))
good_deal.show()

+---------+-------------+---------+
| Supplies|GoodDealRatio|      Tag|
+---------+-------------+---------+
|   AVTech|          1.0|Good Deal|
|TechWorld|          0.0|  Average|
|PrintFast|          1.0|Good Deal|
| FreezeIt|          0.0|  Average|
|  ChairCo|          0.0|  Average|
+---------+-------------+---------+



Scenario 3: Cost Forecasting

In [0]:
#1. Calculate TotalStockValue = StockQty * UnitPrice .
df = df.withColumn("TotalStockValue", col("StockQty") * col("UnitPrice"))

#2. Identify top 3 highest-value items.
df.orderBy(col("TotalStockValue").desc()).select("ItemName", "TotalStockValue").show(3)

#3. Export the result as a Parquet file partitioned by Warehouse .
df.write.mode("overwrite").partitionBy("Warehouse").parquet("output/warehouse_stock_value")

+------------+---------------+
|    ItemName|TotalStockValue|
+------------+---------------+
|      LED TV|        1500000|
|      Laptop|         700000|
|Office Chair|         240000|
+------------+---------------+
only showing top 3 rows



Scenario 4: Warehouse Utilization

In [0]:
# 1. Count items stored per warehouse.
df.groupBy("Warehouse").count().show()

#2. Average stock per category in each warehouse.
df.groupBy("Warehouse", "Category").agg(avg("StockQty").alias("AvgStock")).show()

#3. Determine underutilized warehouses ( total stock < 100 ).
df.groupBy("Warehouse").agg(expr("sum(StockQty)").alias("TotalStock")) \
    .filter("TotalStock < 100").show()

+----------+-----+
| Warehouse|count|
+----------+-----+
|WarehouseA|    2|
|WarehouseC|    1|
|WarehouseB|    2|
+----------+-----+

+----------+-----------+--------+
| Warehouse|   Category|AvgStock|
+----------+-----------+--------+
|WarehouseB|Electronics|     6.5|
|WarehouseA|  Furniture|    40.0|
|WarehouseC| Appliances|     5.0|
|WarehouseA|Electronics|    50.0|
+----------+-----------+--------+

+----------+----------+
| Warehouse|TotalStock|
+----------+----------+
|WarehouseA|        90|
|WarehouseC|         5|
|WarehouseB|        13|
+----------+----------+



Scenario 5: Delta Audit Trail

In [0]:
from delta.tables import DeltaTable

# 1. Save as Delta table retail_inventory .
df.write.format("delta").mode("overwrite").save("/tmp/retail_inventory")

#2. Update stock of 'Laptop' to 20.
delta_table = DeltaTable.forPath(spark, "/tmp/retail_inventory")
delta_table.update("ItemName = 'Laptop'", {"StockQty": "20"})

#3. Delete any item with StockQty = 0 .
delta_table.delete("StockQty = 0")

#4. Run DESCRIBE HISTORY and query VERSION AS OF previous state.
spark.sql("DESCRIBE HISTORY delta.`/tmp/retail_inventory`").show()

spark.read.format("delta").option("versionAsOf", 0).load("/tmp/retail_inventory").show()

+-------+-------------------+----------------+--------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|operation| operationParameters| job|          notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      2|2025-06-19 05:49:53|4028198190791787|azuser3553_mml.lo...|   DELETE|{predicate -> ["(...|NULL|{1052067078041129}|0611-043506-43vn1hs6|          1|WriteSerializable|        false|{numRemovedFiles ...|        NULL|Databricks-Runtim...|
|      1|2025-06-19 05:49:52

Scenario 6: Alerts from Restock Logs

In [0]:
restock_logs = spark.read.option("header", True).option("inferSchema", True).csv("file:/Workspace/Shared/restock_logs.csv")
restock_logs = restock_logs.withColumn("RestockDate", to_date("RestockDate"))

#1. Join with inventory table to update StockQty.
inventory_df = spark.read.format("delta").load("/tmp/retail_inventory")
restock_joined = inventory_df.join(restock_logs, "ItemID", "left")

#2. Calculate new stock and flag RestockedRecently = true for updated items.
updated_df = restock_joined.withColumn(
    "NewStockQty", col("StockQty") + when(col("QuantityAdded").isNull(), 0).otherwise(col("QuantityAdded"))
).withColumn("RestockedRecently", col("QuantityAdded").isNotNull())

updated_df.select("ItemID", "ItemName", "StockQty", "QuantityAdded", "NewStockQty", "RestockedRecently").show()

#3. Use MERGE INTO to update in Delta.
merge_table = DeltaTable.forPath(spark, "/tmp/retail_inventory")

merge_table.alias("target").merge(
    updated_df.alias("source"),
    "target.ItemID = source.ItemID"
).whenMatchedUpdate(set={
    "StockQty": "source.NewStockQty"
}).execute()

+------+------------+--------+-------------+-----------+-----------------+
|ItemID|    ItemName|StockQty|QuantityAdded|NewStockQty|RestockedRecently|
+------+------------+--------+-------------+-----------+-----------------+
|  I002|      Laptop|      30|           10|         40|             true|
|  I001|      LED TV|      70|           20|         90|             true|
|  I005|     Printer|       8|            5|         13|             true|
|  I003|Office Chair|      40|         NULL|         40|            false|
|  I004|Refrigerator|       5|         NULL|          5|            false|
+------+------------+--------+-------------+-----------+-----------------+



Scenario 7: Report Generation with SQL Views

In [0]:
#1. Create SQL view inventory_summary
df.createOrReplaceTempView("inventory_summary_view")
spark.sql("""
    SELECT ItemName, Category, StockQty, NeedsReorder, TotalStockValue
    FROM inventory_summary_view
""").createOrReplaceTempView("inventory_summary")
spark.sql("SELECT * FROM inventory_summary").show()

#2. Create view supplier_leaderboard sorted by average price
df.groupBy("Supplies").agg(avg("UnitPrice").alias("AvgPrice")) \
    .orderBy("AvgPrice").createOrReplaceTempView("supplier_leaderboard")
spark.sql("SELECT * FROM supplier_leaderboard").show()

+------------+-----------+--------+------------+---------------+
|    ItemName|   Category|StockQty|NeedsReorder|TotalStockValue|
+------------+-----------+--------+------------+---------------+
|      LED TV|Electronics|      50|       false|        1500000|
|      Laptop|Electronics|      10|        true|         700000|
|Office Chair|  Furniture|      40|       false|         240000|
|Refrigerator| Appliances|       5|        true|         125000|
|     Printer|Electronics|       3|        true|          24000|
+------------+-----------+--------+------------+---------------+

+---------+--------+
| Supplies|AvgPrice|
+---------+--------+
|  ChairCo|  6000.0|
|PrintFast|  8000.0|
| FreezeIt| 25000.0|
|   AVTech| 30000.0|
|TechWorld| 70000.0|
+---------+--------+



Scenario 8: Advanced Filtering

In [0]:
#1. Use when / otherwise to categorize items
df = df.withColumn("StockStatus", when(col("StockQty") > 2 * col("ReorderLevel"), "Overstocked")
                                   .when(col("StockQty") < col("ReorderLevel"), "LowStock")
                                   .otherwise("Normal"))

df.select("ItemName", "StockQty", "ReorderLevel", "StockStatus").show()

#2. Use .filter() and .where() for the same and compare.
df.filter(col("StockStatus") == "LowStock").show()
df.where("StockStatus = 'Overstocked'").show()

+------------+--------+------------+-----------+
|    ItemName|StockQty|ReorderLevel|StockStatus|
+------------+--------+------------+-----------+
|      LED TV|      50|          20|Overstocked|
|      Laptop|      10|          15|   LowStock|
|Office Chair|      40|          10|Overstocked|
|Refrigerator|       5|          10|   LowStock|
|     Printer|       3|           5|   LowStock|
+------------+--------+------------+-----------+

+------+------------+-----------+----------+--------+------------+-------------+---------+---------+------------+---------------+-----------+
|ItemID|    ItemName|   Category| Warehouse|StockQty|ReorderLevel|LastRestocked|UnitPrice| Supplies|NeedsReorder|TotalStockValue|StockStatus|
+------+------------+-----------+----------+--------+------------+-------------+---------+---------+------------+---------------+-----------+
|  I002|      Laptop|Electronics|WarehouseB|      10|          15|   2024-04-01|    70000|TechWorld|        true|         700000|   

Scenario 9: Feature Engineering

In [0]:
#1. Extract RestockMonth from LastRestocked .
df = df.withColumn("RestockMonth", expr("month(LastRestocked)"))

#2. Create feature: StockAge = CURRENT_DATE - LastRestockede
df = df.withColumn("StockAge", datediff(current_date(), col("LastRestocked")))

#3. Bucket StockAge into: New, Moderate, Stale
df = df.withColumn("StockAgeCategory", when(col("StockAge") < 30, "New")
                                       .when(col("StockAge") < 90, "Moderate")
                                       .otherwise("Stale"))

df.select("ItemName", "LastRestocked", "StockAge", "StockAgeCategory").show()

Scenario 10: Export Options

In [0]:
# 1. Write to CSV, JSON, Delta
df.write.mode("overwrite").csv("export/analysts/inventory.csv", header=True)
df.write.mode("overwrite").json("export/integration/inventory.json")
df.write.format("delta").mode("overwrite").save("/dbfs/export/pipelines/inventory.delta")

# 2. Save with meaningful file and partition names
df.filter("StockStatus = 'Stale'") \
    .write.mode("overwrite") \
    .partitionBy("Category") \
    .parquet("export/inventory/stale_items")